In [16]:
# | default_exp llm/qna

In [34]:
#| exporti
from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.chains import ConversationChain

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory

from IPython.display import clear_output
from langchain_core.output_parsers import StrOutputParser

from langchain.memory import ConversationBufferWindowMemory


In [18]:
# | exporti

llm = ChatOpenAI(
    openai_api_base="http://172.20.2.239:1234/v1/",
    openai_api_key="not-needed",
    model="local_model",
    # callbacks=[StreamingStdOutCallbackHandler()],
)

In [35]:
# | export

def generate_promptchain_for_string(system_prompt, llm) -> ConversationChain:
    parser = StrOutputParser()
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=system_prompt),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    return prompt | llm | parser

In [32]:
# | export  

from typing import Any
from queue import Queue, Empty
from langchain.llms import LlamaCpp
from langchain.callbacks.base import BaseCallbackHandler
from langchain.prompts import PromptTemplate
from threading import Thread


def handle_streaming_ref(stream):
    response = []
    for chunk in stream:
        print(chunk, end="|", flush=True)
        response.append(chunk.content)

        clear_output(wait=True)
        response = "".join(response).strip()

        return response

def get_llm_response(
    user_content,
    llm,
    chat_history = None,
    system_prompt="You are a QandA Bot",
    temperature = 0.7
) ->AIMessage:
    
    chat_history = chat_history or ChatMessageHistory()
    chat_history.add_user_message(user_content)

    chain = generate_promptchain_for_string(system_prompt=system_prompt, llm = llm )

    def task():
        response = llm(chain.stream({"messages": chat_history.messages}, temperature = temperature))
        q.put(job_done)
    
    t = Thread(target=task)
    t.start()
    

    chat_history.add_ai_message(response)
    
q = Queue()
job_done = object()

class QueueCallback(BaseCallbackHandler):
    """Callback handler for streaming LLM responses to a queue."""

    def __init__(self, q):
        self.q = q

    def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        self.q.put(token)

    def on_llm_end(self, *args, **kwargs: Any) -> None:
        return self.q.empty()
    
callbacks = [StreamingStdOutCallbackHandler(), QueueCallback(q)]

llm = ChatOpenAI(
    openai_api_base="http://172.20.2.239:1234/v1/",
    openai_api_key="not-needed",
    model="local_model",
    callbacks=callbacks,
)

In [33]:
get_llm_response(
    user_content ="Translate this sentence from English to French: I love programming.",
    system_prompt="You are a translator.  Be very succinct", 
    llm = llm)


<generator object RunnableSequence.stream at 0x7f97cb9216c0>

In [26]:
#| export
class MemoryManager:
    system_prompt: str
    chat_history = None
    llm

    def __init__(self, system_prompt, llm):
        self.llm = llm
        self.system_prompt = system_prompt
        
        self.chat_history = ChatMessageHistory()   

    def add_user_message(self, content):

        self.chat_history.add_user_message(content)
        self._get_ai_streaming_response()

    def _generate_promptchain(self) -> ConversationChain:
        return generate_promptchain(system_prompt=self.system_prompt, llm = self.llm)

    
    def _get_ai_streaming_response(self):

        chain = self._generate_promptchain()

        response = []
        for chunk in chain.stream({"messages":self.chat_history.messages}):
            print(chunk.content, end="", flush=True)
            response.append(chunk.content)
        
        clear_output(wait=True)
        response = "".join(response).strip()

        self.chat_history.add_ai_message(response)
        return response

In [27]:
mm = MemoryManager(system_prompt="You are a translator.  Be very succinct", llm = llm)

print(mm.add_user_message("Translate this sentence from English to French: I love programming."))


None


In [ ]:
print(mm.add_user_message("repeat what you just said?"))

Sure! In French, the sentence "I love programming" can be translated as "Je préfère programmer" or "J'aime programmer". Both mean "I prefer to program" or "I like to program".


In [ ]:
#+| hide
import nbdev
nbdev.nbdev_export()

In [ ]:
# !jupyter nbconvert --to python llm_qna.ipynb --output ./_test/llm.py